This notebook makes the latex tables for the SNeHa paper

Table A1. Galaxy, Supernova, Type, RA, Dec, In Sample, Reference

A2. SN name, type, Native Resolution, Ha, line width, A_v, HII, Grove identifier, SNR?, CO, metallicity [12+log(O/H) (dex)], 
R_gal (if center of galaxy, can fake rotation being big line width), galactocentric radius.

In [1]:
#import packages and formatting statements
import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
import os


In [2]:
# Compile galaxy and OSC objects
data = Table.read('../Data/1.MUSE_SN_Sample.csv', format = "csv")
data[0:2]


Galaxy,Supernova,Type,SN_ra,SN_dec,In_Sample
str7,str11,str12,float64,float64,str4
NGC1087,SN1995V,II,41.61152777777777,-0.4987861111111111,True
NGC1300,SN2022acko,II,49.91245833333333,-19.39518888888889,True


In [3]:
# Compile intensities/extinctions
ints = Table.read('../Data/1.sn_ints_exts.csv', format = "csv")
ints[0:2]


Supernova,sn_type,int_nat,int_150,int_1kpc,a_v,c_o,e_co,co_snr,co,halpha_lws,gcr,gcr_reff,met_theory
str11,str5,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64
SN1995V,II,8.839672735197591e+39,9.098356574390131e+39,5.626363456901733e+39,1.2788075992834158,10.1257905960083,0.9063823223114014,11.17165470123291,Yes,52.49597118323195,0.14295581272079702,3.396280301797827,8.241260378874152
SN2022acko,II,3.445745101042276e+38,5.543860583782077e+38,7.868984950864679e+38,0.3049447396212661,-0.13982966542243958,0.7117888331413269,0.0,No,66.20645202360187,0.3791372256557472,5.349132803990211,8.194418508484775


In [4]:
# Compile HII region data
hii = Table.read('../Data/1.sn_hii_dists.csv', format = "csv")
hii["sn_name"].info.name = "Supernova"

hii[0:2]

Supernova,sn_type,in_HII,hii_dist,hii_val
str11,str12,str3,float64,int64
SN1995V,II,Yes,0.1,731
SN2022acko,II,No,130.01600444193133,-1


In [5]:
# Compile muse data
muse = Table.read('../Data/0.MUSEdata.csv', format = "csv")
muse_join = join(muse, data, keys=["Galaxy"])
muse_join = join(muse_join, hii, keys = ["Supernova"])
muse_join = join(muse_join, ints, keys = ["Supernova"])
muse_join[0:4]



Galaxy,Distance,Gal_ra,Gal_dec,Pos_ang,Incl,r_eff,MUSE_nat_res,MUSE_phys_res,maps_musedap,maps_nat,maps_15asec,maps_150,maps_1kpc,maps_HII,maps_irac,maps_CO21,maps_eCO21,maps_ext,maps_vdisp,maps_HST,JWST200,JWST300,JWST335,JWST360,JWST770,JWST1000,JWST1130,JWST2100,Supernova,Type,SN_ra,SN_dec,In_Sample,sn_type_1,in_HII,hii_dist,hii_val,sn_type_2,int_nat,int_150,int_1kpc,a_v,c_o,e_co,co_snr,co,halpha_lws,gcr,gcr_reff,met_theory
str7,float64,float64,float64,float64,float64,float64,float64,int64,str71,str58,str60,str64,str63,str50,str40,str72,str73,str36,str32,str45,str93,str93,str93,str93,str93,str94,str94,str94,str11,str12,float64,float64,str4,str12,str3,float64,int64,str5,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64
NGC1566,17.69,65.00159,-54.93801,214.7,29.5,36.968307,0.8,80,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1566-0.80asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1566_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC1566_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1566_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1566_sfr_1kpc.fits,../Data/HII_reg_masks/NGC1566_nebulae_mask_V2.fits,../Data/IRAC/ngc1566_irac1_atGauss4.fits,../../GalaxyFiles/ALMACO2-1/ngc1566_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1566_12m+7m+tp_co21_150pc_broad_emom0.fits,../Data/SFR_v1/NGC1566_MAPS_sfr.fits,../Data/vdisp/ngc1566_vdisp.fits,../../GalaxyFiles/HST/ngc1566_hst_ha_sic.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F200W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F300W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F330W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F365W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F770W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F1000W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F1130W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1566_F2100W_atGauss7p5_anchored.fits,ASASSN-14ha,II,65.0058875,-54.93807777777778,True,II,Yes,0.1,877,II,6.206906305070956e+39,1.2762259220392566e+40,5.346720109001179e+39,0.8637144049713334,61.47018814086914,1.4010099172592163,43.87562561035156,Yes,57.169856446603866,0.04834469069591773,1.3077334240899299,8.564613863308672
NGC4321,15.21,185.72887,15.822304,156.2,38.5,74.58043,1.16,43,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC4321-1.16asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC4321_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/15asec/NGC4321_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC4321_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC4321_sfr_1kpc.fits,../Data/HII_reg_masks/NGC4321_nebulae_mask_V2.fits,../Data/IRAC/ngc4321_irac1_atGauss4.fits,../../GalaxyFiles/ALMACO2-1/ngc4321_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc4321_12m+7m+tp_co21_150pc_broad_emom0.fits,../Data/SFR_v1/NGC4321_MAPS_sfr.fits,../Data/vdisp/ngc4321_vdisp.fits,../../GalaxyFiles/HST/ngc4321_hst_ha_sic.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F200W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F300W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F330W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F365W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F770W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F1000W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc4321_F1130W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_G

In [6]:
# prep ra and dec, int and a_v values for table columns
ra   = [float(x) for x in muse_join["SN_ra"]]
ra   = [round(x,4) for x in ra]
dec  = [float(x) for x in muse_join["SN_dec"]]
dec  = [round(x,4) for x in dec]   
int_nat  = [round(x/(10**37)) for x in muse_join["int_nat"]]  
a_v  = [round(x,2) for x in muse_join["a_v"]]  
r_kpc = [round(x,2) for x in muse_join["gcr"]]  
r_eff = [round(x,2) for x in muse_join["gcr_reff"]]
lws = [round(x) for x in muse_join["halpha_lws"]]

In [7]:
# prep in sample and paper columns 
inSample, paper = [],[]
for i in range(len(muse_join["In_Sample"])):
    paper.append("?")
    if muse_join["In_Sample"][i] == "True":
        inSample.append("\checkmark")
    else: 
        inSample.append("-")  

In [8]:
# prep native resolution column
nat_res = []
for i in range(len(muse_join)):
    nat_res.append(str(muse_join["MUSE_nat_res"][i]) + " (" + str(muse_join["MUSE_phys_res"][i]) + ")")


In [9]:
# prep line width and BPT columns
hdu = pyfits.open('../Data/0.Nebulae_catalogue_public.fits')
header = hdu[1].header
neb = Table(hdu[1].data)
neb[0:111]

gal_name,region_ID,cen_ra,cen_dec,flag_edge,flag_star,deproj_phi,region_area,HB4861_FLUX,OIII5006_FLUX,HA6562_FLUX,NII6583_FLUX,SII6716_FLUX,SII6730_FLUX,SIII9068_FLUX,OI6300_FLUX,HB4861_FLUX_ERR,OIII5006_FLUX_ERR,HA6562_FLUX_ERR,NII6583_FLUX_ERR,SII6716_FLUX_ERR,SII6730_FLUX_ERR,SIII9068_FLUX_ERR,OI6300_FLUX_ERR,HA6562_SIGMA,NII6583_SIGMA,OIII5006_SIGMA,HA6562_VEL,NII6583_VEL,OIII5006_VEL,HA6562_SIGMA_ERR,NII6583_SIGMA_ERR,OIII5006_SIGMA_ERR,HA6562_VEL_ERR,NII6583_VEL_ERR,OIII5006_VEL_ERR,HB4861_FLUX_CORR,OIII5006_FLUX_CORR,HA6562_FLUX_CORR,NII6583_FLUX_CORR,SII6716_FLUX_CORR,SII6730_FLUX_CORR,SIII9068_FLUX_CORR,OI6300_FLUX_CORR,HB4861_FLUX_CORR_ERR,OIII5006_FLUX_CORR_ERR,HA6562_FLUX_CORR_ERR,NII6583_FLUX_CORR_ERR,SII6716_FLUX_CORR_ERR,SII6730_FLUX_CORR_ERR,SIII9068_FLUX_CORR_ERR,OI6300_FLUX_CORR_ERR,met_scal,met_scal_err,BPT_NII,BPT_SII,BPT_OI,Environment,Delta_met_scal,deproj_r_R25,deproj_r_reff,EW_HA6562_raw,EW_HA6562_raw_ERR,EW_HB4861_raw,EW_HB4861_raw_ERR,EW_HA6562_fit,EW_HA6562_fit_ERR,EW_HB4861_fit,EW_HB4861_fit_ERR,logU,logU_err,AV,AV_ERR,HA6562_LUM_CORR
str32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC5332,1.0,353.6217889985926,-36.086205660984305,0.0,0.0,0.6454405340574735,32.0,1539.2341650021465,1219.0416061730582,4121.1986373108,1476.084225361972,792.9254230583941,553.7915648041692,0.0,409.1388314208345,72.04334579496953,67.78753986403167,63.36012574499638,42.45388074223228,43.89198739851983,42.96715942103822,52.852537047392694,45.25961484572321,29.355670183624387,20.60076596820052,1.0000000000004547,16.62758507329847,22.17945502633234,6.350641875665667,2.1354674718708075,4.301285069747231,336.6408968897181,1.1122747085471123,1.7740315531493769,5.761855482475623,1539.2341650021465,1219.0416061730582,4121.1986373108,1476.084225361972,792.9254230583941,553.7915648041692,0.0,409.1388314208345,72.04334579496953,67.78753986403167,63.36012574499638,42.45388074223228,43.89198739851983,42.96715942103822,nan,45.25961484572321,nan,nan,0.0,0.0,3.0,9.0,nan,0.3415751491467562,0.7527512481926566,6.860629680858027,0.21143634038481388,-1.3213760584367453,-0.40228188757079336,9.520132003761226,0.4556576542009083,3.263058487334211,0.3460152144937209,nan,nan,0.0,0.0,4.002986321046122e+35
IC5332,2.0,353.6220839769834,-36.12298517170686,0.0,0.0,3.1430314809034154,30.0,2971.16135165289,0.0,11767.886277696742,177.89097938247303,568.4848741059337,448.33009305663313,0.0,304.2578829553127,76.31829329400553,45.715872080199546,82.95389957751453,35.15246251772309,35.367342635693554,35.78791264031514,38.21000194263814,33.13318787181186,105.02138505236137,27.113437433911812,1.0000000000004547,36.97313212960351,16.568530277047007,-15.393545817169487,1.0243705674981567,7.041511550500124,4.46662976546243e-14,0.9423574644351665,3.2837290437957396,8.322472509673083e-15,8423.199515503842,0.0,24093.228116320577,363.2978291923634,1142.8672790660721,899.8265115207142,0.0,643.3843758861905,749.9248669139744,nan,1422.4891311834986,74.86105301088647,96.52990610516578,88.25693388335675,nan,80.38925769131231,nan,nan,-1.0,-1.0,-1.0,9.0,nan,0.5034968633355311,1.1095886023431683,39.798272349087,0.45594507917029153,2.968920806852992,0.40770707886767193,41.690107905697566,2.2121216975076936,6.60450569902629,0.5214672641223538,nan,nan,0.9574486905172007,0.07832516526534104,2.340213881134546e+36
IC5332,3.0,353.5993617626245,-36.083693581872964,0.0,0.0,5.940899821301457,65.0,10503.734762881732,3182.033751213841,54548.91485705733,460.

In [10]:
# prep nebular columns, match on region ID
bpt_nii, bpt_sii, bpt_oi, met,r25 = [],[],[],[],[]
for i in range(len(muse_join)):
    idxID = np.where(neb["region_ID"] == muse_join["hii_val"][i]) # find unique region index
    idxGal = np.where(neb["gal_name"] == muse_join["Galaxy"][i])  # for just the one galaxy
    idx = np.intersect1d(idxID, idxGal)
    if len(idx) > 0:
        bpt_nii.append(neb["BPT_NII"][idx][0])
        bpt_sii.append(neb["BPT_SII"][idx][0])
        bpt_oi.append(neb["BPT_OI"][idx][0])
#         met.append(str(round(neb["met_scal"][idx][0],2)))
        met.append(str(round(muse_join["met_theory"][i],2)) + " ("+ str(round(neb["met_scal"][idx][0],2)) + ")")
        r25.append(round(neb["deproj_r_R25"][idx][0],2))
    else:
        bpt_nii.append("-")
        bpt_sii.append("-")
        bpt_oi.append("-")
        met.append(str(round(muse_join["met_theory"][i],2)))
        r25.append("-")


In [11]:
# make column for HII region identifier
hii_id = []
for i in range(len(muse_join)):
    hii_str = muse_join["in_HII"][i] + " (" + str(muse_join["hii_val"][i]) + ")"
    hii_id.append(hii_str)
    

In [12]:
# make Table for Latex Table
texTable = Table({"Galaxy"    : data["Galaxy"],
                  "Supernova" : data["Supernova"],
                  "Type"      : data["Type"],
                  "Ra"        : ra,
                  "Dec"       : dec,
                  "In Sample" : inSample,
                  "Reference" : paper})

In [13]:
ascii.write(texTable, '../Data/2.SNeHa_objects_table.tex', format='latex', fast_writer=False, overwrite = True)  

In [14]:
snr = np.empty(len(muse_join),dtype=str)
snr.fill("-")

texTable = Table({"Supernova"  : muse_join["Supernova"],
                  "Type"       : muse_join["Type"],
                  "NatRes"     : nat_res,
                  "Halpha"     : int_nat,
                  "lineWidth"  : lws,
                  "A_v"        : a_v,
                  "HII"        : hii_id,
                  "BPT_NII"    : bpt_nii,
                  "BPT_SII"    : bpt_sii,
                  "BPT_OI"     : bpt_oi,
#                   "SNR"        : snr,
                  "CO"         : muse_join["co"],
                  "Metal"      : met,
                  "R_gal"      : r_kpc,
                  "R_eff"      : r_eff})

ascii.write(texTable, '../Data/2.SNeHa_measurements_table.tex', format='latex', fast_writer=False, overwrite = True)  


In [15]:
texTable

Supernova,Type,NatRes,Halpha,lineWidth,A_v,HII,BPT_NII,BPT_SII,BPT_OI,CO,Metal,R_gal,R_eff
str11,str12,str10,int64,int64,float64,str10,str32,str32,str32,str3,str11,float64,float64
ASASSN-14ha,II,0.8 (80),621,57,0.86,Yes (877),0.0,0.0,0.0,Yes,8.56 (8.61),0.05,1.31
SN1901B,I,1.16 (43),211,53,0.92,No (-1),-,-,-,Yes,8.38,0.57,7.67
SN1926A,II,0.78 (73),82,67,0.0,No (-1),-,-,-,No,8.34,0.35,8.42
SN1957C,Unclassified,1.15 (76),294,63,0.38,No (-1),-,-,-,No,8.52,0.81,43.17
SN1959E,I,1.16 (43),361,55,1.07,No (-1),-,-,-,Yes,8.48,0.3,3.98
SN1961I,II,0.78 (73),415,58,0.35,Yes (373),0.0,0.0,0.0,No,8.3 (8.56),0.4,9.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN2017gax,Ib/c,0.96 (104),419,501,0.0,Yes (429),-1.0,-1.0,-1.0,Yes,8.44 (nan),0.36,9.98
SN2019ehk,Ib,1.16 (43),104,76,1.47,Yes (1722),1.0,0.0,0.0,No,8.55 (nan),0.12,1.59
